In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
# Helper

# Init weights

def init_weights(shape):
    init_random_dist = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(init_random_dist)

# Init bias

def init_bias(shape):
    init_bias_vals = tf.constant(0.1, shape=shape)
    return tf.Variable(init_bias_vals)

# Conv2d

def conv2d(x, W):
    # x --> [batch, H,W,Channels]
    # W --> [filter H, filter W, Channels In, Channels out]

    return tf.nn.conv2d(x,W,strides=[1,1,1,1], padding='SAME')

# Pooling

def max_pool_2by2(x):
    # x --> [batch, h,w,c]
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


In [4]:
# Conv Layer
def convolational_layer(input_x, shape):
    W = init_weights(shape)
    b = init_bias([shape[3]])
    return tf.nn.relu(conv2d(input_x,W)+b)

# Normal layer

def normal_full_layer(input_layer, size):
    input_size = int(input_layer.get_shape()[1])
    W = init_weights([input_size, size])
    b = init_bias([size])
    return tf.matmul(input_layer, W) + b

In [5]:
# Placeholders
x = tf.placeholder(tf.float32, shape=[None, 784])

In [6]:
y_true = tf.placeholder(tf.float32, shape=[None, 10])

In [7]:
# Layers
x_image = tf.reshape(x, [-1, 28, 28,1])

In [8]:
convo_1 = convolational_layer(x_image, shape=[5,5,1,32])
convo_1_pooling = max_pool_2by2(convo_1)

In [9]:
convo_2 = convolational_layer(convo_1_pooling, shape=[5,5,32,64])
convo_2_pooling = max_pool_2by2(convo_2)

In [10]:
convo_2_flat = tf.reshape(convo_2_pooling, [-1,7*7*64])
full_layer_one = tf.nn.relu(normal_full_layer(convo_2_flat, 1024))

In [11]:
# Dropout
hold_prob = tf.placeholder(tf.float32)
full_one_dropout = tf.nn.dropout(full_layer_one, keep_prob=hold_prob)

In [12]:
y_pred = normal_full_layer(full_one_dropout, 10)

In [13]:
# Loss function
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=y_true, logits=y_pred))

In [14]:
# Optimizer
opt = tf.train.AdamOptimizer(learning_rate=0.001)
train = opt.minimize(cross_entropy)

init = tf.global_variables_initializer()

In [ ]:
steps = 5000

with tf.Session() as sess:
    
    sess.run(init)
    
    for i in range(steps):
        
        batch_x, batch_y = mnist.train.next_batch(50)
        
        sess.run(train, feed_dict={x:batch_x, y_true:batch_y, hold_prob:0.5})
        
        if i%100 == 0:
            
            print("On step {}".format(i))
            print("Accuracy: ")
            matches = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
            
            acc = tf.reduce_mean(tf.cast(matches, tf.float32))
            
            print(sess.run(acc, feed_dict={x:mnist.test.images, y_true: mnist.train.labels, hold_prob:1.0}))
            print('\n')